# PROYECTO DATOS CLIMA BOGOTA POR LOCALIDADES
#### Sebastian Abella Rocha
#### Luisa Magaly Achury
#### Johan Cuellar Orjuela
##### Fecha 10 enero 2024

En este cuaderno de jupyter se realiza un flujo de trabajo para clasificar la mala calidad del clima en localidades de Bogotá

### Importar librerías

In [ ]:
import numpy as np
import pandas as pd

### Cargar datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dataini = pd.read_csv('historico_siniestros_bogota_d.c_-.csv')
dataini = pd.read_csv('/content/drive/MyDrive/data/reporte_estaciones_clima_3.csv', sep=';', encoding='latin-1')

In [ ]:
dataini.head()

,DateTime,PM10,PM2.5,CO,Vel Viento,Dir Viento,Temperatura,Precipitacion,HR,PM2.5 Flow,Localidad
0,31-12-2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kennedy
1,01-01-2006,"125,5",NaN,NaN,"1,2",24,NaN,0,NaN,NaN,Kennedy
2,02-01-2006,"100,8",NaN,NaN,"1,4",7,NaN,"3,1",NaN,NaN,Kennedy
3,03-01-2006,"60,5",NaN,NaN,1,17,NaN,"5,9",NaN,NaN,Kennedy
4,04-01-2006,NaN,NaN,NaN,1,22,NaN,"0,1",NaN,NaN,Kennedy


In [ ]:
dataini.shape

(18590, 11)

In [ ]:
# FUNCIONES PARA MEDIR LA CALIDAD DEL AIRE
# Definir funciones para calcular la calidad del aire
def asignar_puntuacion_pm(valor):
    if valor <= 50:
        return 0
    elif valor <= 100:
        return 50
    elif valor <= 150:
        return 100
    elif valor <= 200:
        return 150
    elif valor <= 300:
        return 200
    else:
        return 300

def asignar_puntuacion_co(valor):
    if valor <= 4.4:
        return 0
    elif valor <= 9.4:
        return 50
    elif valor <= 12.4:
        return 100
    elif valor <= 15.4:
        return 150
    elif valor <= 30.4:
        return 200
    else:
        return 300

def asignar_puntuacion_viento(velocidad):
    if velocidad <= 5:
        return 0
    elif velocidad <= 10:
        return 50
    else:
        return 100

def asignar_puntuacion_temperatura(valor):
    if valor <= 25:
        return 0
    elif valor <= 30:
        return 50
    elif valor <= 35:
        return 100
    elif valor <= 40:
        return 150
    else:
        return 200

def calcular_indice_calidad_aire(pm10, pm25, co, velocidad_viento, temperatura):
    puntuacion_pm10 = asignar_puntuacion_pm(pm10)
    puntuacion_pm25 = asignar_puntuacion_pm(pm25)
    puntuacion_co = asignar_puntuacion_co(co)
    puntuacion_viento = asignar_puntuacion_viento(velocidad_viento)
    puntuacion_temperatura = asignar_puntuacion_temperatura(temperatura)

    indice_total = puntuacion_pm10 + puntuacion_pm25 + puntuacion_co + puntuacion_viento + puntuacion_temperatura

    return indice_total

def interpretar_calidad_aire(indice_total):
    if indice_total <= 50:
        return 1
    elif indice_total <= 100:
        return 2
    elif indice_total <= 150:
        return 3
    elif indice_total <= 200:
        return 4
    else:
        return 5

# Definir la función para calcular la calidad del aire
def calcular_calidad_aire(row):
    pm10 = row['PM10']
    pm25 = row['PM2.5']
    co = row['CO']
    velocidad_viento = row['Vel Viento']
    temperatura = row['Temperatura']

    # Verificar si alguno de los valores es NaN
    if any(pd.isna(x) for x in [pm10, pm25, co, velocidad_viento, temperatura]):
        return "No Definido"

    # Verificar si alguno de los valores es nulo o no definido
    if pd.isnull(pm10) or pd.isnull(pm25) or pd.isnull(co) or pd.isnull(velocidad_viento) or pd.isnull(temperatura):
        return "No definido"

    # Calcular la calidad del aire
    calidad_aire = calcular_indice_calidad_aire(pm10, pm25, co, velocidad_viento, temperatura)
    return interpretar_calidad_aire(calidad_aire)

# conversion a decimales
def convertir_reemplazar(valor):
    try:
        # Intentar convertir a float y reemplazar comas por puntos
        return float(str(valor).replace(',', '.'))
    except ValueError:
        # En caso de error, devolver el valor original
        return valor

In [ ]:
data = dataini

# Convertir columnas a decimal
data['Vel Viento'] = data['Vel Viento'].apply(convertir_reemplazar).astype(float)
data['PM10'] = data['PM10'].apply(convertir_reemplazar).astype(float)
data['PM2.5'] = data['PM2.5'].apply(convertir_reemplazar).astype(float)
data['CO'] = data['CO'].apply(convertir_reemplazar).astype(float)
data['Temperatura'] = data['Temperatura'].apply(convertir_reemplazar).astype(float)

# Agregar un promedio
promedio_vel_viento = data['Vel Viento'].mean()
promedio_temp = data['Temperatura'].mean()
promedio_pm = data['PM2.5'].mean()

data['Vel Viento'] = data['Vel Viento'].apply(convertir_reemplazar).astype(float).replace(np.nan, promedio_vel_viento)
data['Temperatura'] = data['Temperatura'].apply(convertir_reemplazar).astype(float).replace(np.nan, promedio_temp)
data['PM2.5'] = data['PM2.5'].apply(convertir_reemplazar).astype(float).replace(np.nan, promedio_pm)


# Aplicar la función a cada fila y crear la nueva columna 'Calidad'
data['Calidad'] = data.apply(calcular_calidad_aire, axis=1)

# Filtrar valores deseados
calidad_deseados = [1, 2, 3, 4, 5]
localidades_deseadas = ['Ciudad Bolívar', 'Usme', 'Kennedy', 'Puente Aranda', 'Suba']
data = data[(data['Calidad'].isin(calidad_deseados))& (data['Localidad'].isin(localidades_deseadas))]

# Eliminar registros no utilizados
data = data.drop(['Dir Viento', 'Precipitacion', 'HR', 'PM2.5 Flow'], axis=1)
data.head()

,DateTime,PM10,PM2.5,CO,Vel Viento,Temperatura,Localidad,Calidad
697,29-11-2007,105.0,21.900000,3.3,1.3,14.36835,Kennedy,2
698,30-11-2007,70.8,23.149158,2.9,1.4,14.36835,Kennedy,1
699,01-12-2007,84.4,23.149158,3.0,1.4,14.36835,Kennedy,1
700,02-12-2007,44.5,23.149158,2.8,1.4,14.36835,Kennedy,1
701,03-12-2007,51.8,23.149158,3.0,1.0,14.36835,Kennedy,1


In [ ]:
data.shape

(9533, 8)

In [ ]:
# Convertir la columna 'DateTime' a tipo datetime
data['DateTime'] = pd.to_datetime(data['DateTime'], format='%d-%m-%Y')

# Extraer el año y asignarlo a una nueva columna 'Año'
data['Año'] = pd.DatetimeIndex(data['DateTime']).year

# Eliminar DateTime
data = data.drop(['DateTime'], axis=1)
data.head()

,PM10,PM2.5,CO,Vel Viento,Temperatura,Localidad,Calidad,Año
697,105.0,21.900000,3.3,1.3,14.36835,Kennedy,2,2007
698,70.8,23.149158,2.9,1.4,14.36835,Kennedy,1,2007
699,84.4,23.149158,3.0,1.4,14.36835,Kennedy,1,2007
700,44.5,23.149158,2.8,1.4,14.36835,Kennedy,1,2007
701,51.8,23.149158,3.0,1.0,14.36835,Kennedy,1,2007


In [ ]:
data.shape
# Descargar datos
data.to_csv('/content/drive/MyDrive/data/archivo_con_calidad_new_1.3.0.csv', index=False)


In [ ]:
unique_years = data['Año'].unique()
print("Años encontrados:", unique_years)


Años encontrados: [2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020
 2021 2006]


# Desarrollo por mala calidad aire en las localidades





In [12]:
calidad_deseados = [2,3,4,5]
año_no_deseado = [2009]

data = data[(data['Calidad'].isin(calidad_deseados))& (data['Año'] != 2009)]
data.to_csv('/content/drive/MyDrive/data/archivo_con_calidad_mala_numbers.csv', index=False)

data.head()

,PM10,PM2.5,CO,Vel Viento,Temperatura,Localidad,Calidad,Año
697,105.0,21.9,3.3,1.3,14.36835,Kennedy,2,2007
709,104.3,21.6,3.6,1.2,14.36835,Kennedy,2,2007
712,107.5,24.1,3.6,0.9,14.36835,Kennedy,2,2007
718,100.7,23.0,2.6,1.1,14.36835,Kennedy,2,2007
720,106.3,22.5,2.8,0.8,14.36835,Kennedy,2,2007


In [ ]:
data.shape

(744, 8)

In [ ]:
# getdummies convierte las variable de string a numerico transfomandolas en columnas por cada categoria
data = pd.get_dummies(data, columns=['Calidad','Localidad'])
data.head()

,PM10,PM2.5,CO,Vel Viento,Temperatura,Año,Calidad_Alto,Calidad_Peligroso,Calidad_Regular,Localidad_Kennedy,Localidad_Puente Aranda,Localidad_Suba
758,171.1,23.149158,1.2,1.2,14.36835,2008,0,0,1,1,0,0
764,145.1,50.800000,1.1,1.6,14.36835,2008,0,0,1,1,0,0
776,104.5,39.500000,4.8,1.2,14.36835,2008,0,0,1,1,0,0
787,157.2,23.149158,1.3,2.4,14.36835,2008,0,0,1,1,0,0
1458,126.8,67.100000,2.2,2.5,13.20000,2010,0,0,1,1,0,0


In [ ]:
data.dtypes

PM10                       float64
PM2.5                      float64
CO                         float64
Vel Viento                 float64
Temperatura                float64
Año                          int64
Calidad_Alto                 uint8
Calidad_Peligroso            uint8
Calidad_Regular              uint8
Localidad_Kennedy            uint8
Localidad_Puente Aranda      uint8
Localidad_Suba               uint8
dtype: object

In [ ]:
data['Año'] = pd.Categorical(data['Año'])
categories = data.Año.cat.categories
codes = data.Año.cat.codes

In [ ]:
categories

Int64Index([2008, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2018, 2020], dtype='int64')

In [ ]:
data['Año'] = data.Año.cat.codes
data.dtypes

PM10                       float64
PM2.5                      float64
CO                         float64
Vel Viento                 float64
Temperatura                float64
Año                           int8
Calidad_Alto                 uint8
Calidad_Peligroso            uint8
Calidad_Regular              uint8
Localidad_Kennedy            uint8
Localidad_Puente Aranda      uint8
Localidad_Suba               uint8
dtype: object

In [ ]:
X = data.drop('Año', axis=1)
X.shape

(744, 11)

In [ ]:
Y=pd.DataFrame()
Y['Año'] = data['Año']
Y.shape

(744, 1)

In [ ]:
print(data['Año'].value_counts(normalize=True).mul(100).round(1).astype(str)+'%')

5    32.8%
8    30.2%
9    23.8%
1     6.2%
2     1.7%
3     1.7%
7     1.6%
6     0.9%
0     0.5%
4     0.4%
Name: Año, dtype: object


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Media cero y varianza unitaria
sc = StandardScaler()
Xsc = sc.fit(X)
Xt = Xsc.transform(X)
Xt.shape

(744, 11)

In [ ]:
from sklearn.model_selection import train_test_split as split

In [ ]:
## # Definir función para particionar el conjunto de datos

X_train, X_test, y_train, y_test = split(Xt,Y,
                                         test_size=0.30,
                                         random_state=42,
                                         stratify=Y)

# X_train, X_valid, y_train, y_valid = split(X_train, y_train,
#                                            test_size=0.2,
#                                            stratify=y_train)

## Modelos Supervisados

### Árboles de decisión

In [ ]:
# import xgboost as xgb
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Definición del modelo
arbol1 = DecisionTreeClassifier()

#Entrenamiento y evaluación del modelo
arbol1 = arbol1.fit(X_train,y_train)

# Calcular métricas de desempeño
y_pred = arbol1.predict(X_test)
print("\n", metrics.classification_report(y_test, y_pred, digits=2))

# Visualizar matriz de confusión
# Y_pred13 = np_utils.to_categorical(y_pred13)
# cm = matriz_confusion(Y_test, y_pred, 'si', 'Matriz de confusión clasificador AD')


               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.62      0.71      0.67        14
           2       0.33      0.50      0.40         4
           3       0.50      0.25      0.33         4
           4       0.00      0.00      0.00         1
           5       0.73      0.81      0.77        73
           6       0.00      0.00      0.00         2
           7       1.00      0.50      0.67         4
           8       0.51      0.47      0.49        68
           9       0.47      0.45      0.46        53

    accuracy                           0.58       224
   macro avg       0.42      0.37      0.38       224
weighted avg       0.57      0.58      0.57       224



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Definición del modelo
rfc = RandomForestClassifier(n_estimators=200) # 100 , 200 , 300 , 400 , 500

#Entrenamiento y evaluación del modelo
rfc.fit(X_train, y_train)

# Calcular métricas de desempeño
y_pred2 = rfc.predict(X_test)
print("\n", metrics.classification_report(y_test, y_pred2, digits=2))

<ipython-input-457-4a98ca3b9efa>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)



               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.60      0.86      0.71        14
           2       0.20      0.25      0.22         4
           3       1.00      0.25      0.40         4
           4       0.00      0.00      0.00         1
           5       0.72      0.81      0.76        73
           6       0.00      0.00      0.00         2
           7       0.75      0.75      0.75         4
           8       0.58      0.66      0.62        68
           9       0.85      0.53      0.65        53

    accuracy                           0.67       224
   macro avg       0.47      0.41      0.41       224
weighted avg       0.68      0.67      0.66       224



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Xgboost

In [ ]:
# from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb

In [ ]:
#Definición del modelo
xg_class = xgb.XGBClassifier(objective ='multi:softprob', colsample_bytree = 1, learning_rate = 0.04,
                max_depth = 4, alpha = 10, n_estimators = 200, eval_metric='mlogloss')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#Entrenamiento del modelo
xg_class.fit(X_train, y_train,
             eval_set=[(X_train, y_train), (X_test, y_test)],
             verbose=False)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# Calcular métricas de desempeño
y_pred4 = xg_class.predict(X_test)
print("\n", metrics.classification_report(y_test, y_pred4, digits=2))


               precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.00      0.00      0.00         4
           2       0.69      0.58      0.63        31
           3       0.42      0.41      0.42        87
           4       0.44      0.54      0.48        97
           5       0.36      0.42      0.39       102
           6       0.22      0.24      0.23       100
           7       0.68      0.77      0.72        97
           8       0.44      0.45      0.44       102
           9       0.27      0.26      0.26        88
          10       0.24      0.12      0.16        92
          11       0.23      0.23      0.23        87
          12       0.67      0.21      0.32        28
          13       0.55      0.53      0.54       102
          14       0.69      0.74      0.71       155

    accuracy                           0.45      1178
   macro avg       0.45      0.43      0.43      1178
weighted avg       0.44 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
